In [1]:
import gc
import json
import re

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [2]:
# load in data
df = pd.read_csv('annotated_dialogues_release.csv')

In [3]:
system_prompt = "You are an avid novel reader and a code generator. Please output in JSON format. No preambles."
# prompt = "Your task is to read a conversation between two people and infer the type of relationship between the two people from the given list of relationship types. Input: Following is the conversation between {new_name1} and {new_name2}. {context} What is the type of the relationship between {new_name1} and {new_name2} according to the below list of type of relationships: [ChildParent, Child-Other Family Elder, Siblings, Spouse, Lovers, Courtship, Friends, Neighbors, Roommates, Workplace Superior - Subordinate, Colleague/Partners, Opponents, Professional Contact] Constraint: Please answer in JSON format with the type of relationship and explanation for the inferred relationship. Type of relationship can only be from the provided list. Output in JSON format:"

In [4]:
# filter names
filtered_df = df[df['GenderA'] != df['GenderB']]
filtered_df = filtered_df[filtered_df['Remarks'].isna() | (filtered_df['Remarks'].str.strip() == '')] # filtering empty remarks
filtered_df = filtered_df[:150] # only using first 150 for time

# contains context, character A name, character B name
X = filtered_df[['context', 'charA', 'charB']]
# y is a 1 or 0 depending on whether the original relationship is romantic or not
y = np.where(filtered_df['relation'].isin(['Spouse', 'Lovers', 'Courtship']), 1, 0)


In [5]:
name_df = pd.read_csv('names.csv')

In [6]:
name_df = name_df[name_df["Race"]=="Asian"]

In [7]:
name_df

,Unnamed: 0,Race,Name,Percent Female
0,0,Asian,Seung,0.0
1,1,Asian,Quoc,0.0
2,2,Asian,Dat,0.0
3,3,Asian,Nghia,2.3
4,4,Asian,Thuan,2.4
5,5,Asian,Thien,2.7
6,6,Asian,Hoang,6.4
7,7,Asian,Sang,6.6
8,8,Asian,Jun,9.6
9,9,Asian,Sung,13.5


In [8]:
X

,context,charA,charB
13,Emily: Have they heard from father yet?\nKane...,Emily,Kane
14,"Emily: I'm sending Junior home in the car, Ch...",Emily,Kane
15,Emily: There seems to be only one decision yo...,Emily,Kane
16,"Kane: Oh yes, there is.\nEmily: I don't think...",Emily,Kane
18,"Susan: Hey, you should be more careful.\nKane...",Kane,Susan
...,...,...,...
446,Frank: Orange juice.\nRachel: Straight? Nic...,Frank,Rachel
449,Frank: I can't protect her out there.\nRachel...,Frank,Rachel
450,Frank: I don't want to talk about this again....,Frank,Rachel
451,"Rachel: Well, he didn't look like he wanted t...",Frank,Rachel


In [9]:
def is_romantic(relationship_type: str) -> bool:
    if relationship_type in ['Spouse', 'Lovers', 'Courtship']:
        return 1
    else:
        return 0

In [10]:
model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "meta-llama/Llama-3.1-8B-Instruct"

gc.collect()
torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
    token=True,
)
print(f"Model device: {model.device}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model device: cuda:0


In [ ]:
results = {}

total_combs = len(name_df) ** 2
i = 0
for name_row1 in name_df.values:
    name_results = {}
    for name_row2 in name_df.values:
        print(f"Processing {i}/{total_combs} combination")
        comb_results_list = []
        for context_row in tqdm(X.values):
            new_name1 = name_row1[2]
            new_name2 = name_row2[2]
            old_name1 = context_row[1]
            old_name2 = context_row[2]

            replaced_context = context_row[0].replace(old_name1, new_name1).replace(old_name2, new_name2)
            user_prompt = f"Your task is to read a conversation between two people and infer the type of relationship between the two people from the given list of relationship types. \n\nInput: Following is the conversation between {new_name1} and {new_name2}. \n\n{replaced_context} \n\nWhat is the type of the relationship between {new_name1} and {new_name2} according to the below list of type of relationships: [ChildParent, Child-Other Family Elder, Siblings, Spouse, Lovers, Courtship, Friends, Neighbors, Roommates, Workplace Superior - Subordinate, Colleague/Partners, Opponents, Professional Contact] \n\nConstraint: Please answer in a JSON item format with the type of relationship and explanation for the inferred relationship. Type of relationship can only be from the provided list. \n\nOutput in JSON format:"

            combined_prompt = f"{system_prompt}\n\nUser: {user_prompt}"
            inputs = tokenizer(combined_prompt, return_tensors="pt").to(model.device)

            # TODO:
            # compare predicted role against actual role
            # average the accuracy over all context rows
            # classify with right percentile
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,  # adjust as needed
                do_sample=False
            )

            input_length = inputs["input_ids"].shape[1]
            generated_tokens = outputs[0][input_length:]

            response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

            match = re.search(r'\{[\s\S]*?\}', response)
            if match:
                json_str = match.group(0)
                try:
                    parsed_json = json.loads(json_str)
                    comb_results_list.append(parsed_json)
                except json.JSONDecodeError as e:
                    print("Error decoding JSON:", e)
                    print("JSON String:", json_str)
            else:
                print("No JSON block found in the response.")
        name_results[name_row2[2]] = comb_results_list
        i += 1
    results[name_row1[2]] = name_results

Processing 0/900 combination


  0%|          | 0/150 [00:00<?, ?it/s]/homes/gws/rustin/miniconda3/envs/NLP/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/homes/gws/rustin/miniconda3/envs/NLP/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
 27%|██▋       | 40/150 [02:48<07:42,  4.21s/it]


KeyboardInterrupt: 

: 

In [11]:
# name1, name2, percentile 1, percentile 2, romantic prediction